In [1]:
!pip install neo4j jsonlines


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 4.1 MB/s eta 0:00:00 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.20.0-py3-none-any.whl size=280771 sha256=774136f1a556d711f648ef44c85974ece5790d262e2d67baaa66af1f60229fb0
  Stored in directory: /Users/julian/Library/Caches/pip/wheels/be/66/e9/7348deb6bbcb4e6ba66e6ce85ded4d181d0ac849cc4c94ba9a
Successfully built neo4j
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update,

In [8]:
import jsonlines
from neo4j import GraphDatabase

# Connect to Neo4j
uri = "bolt://localhost:7687"
username = "neo4j"
password = "12345678"
driver = GraphDatabase.driver(uri, auth=(username, password))

def sanitize_label(label):
    # Replace spaces and special characters with underscores
    return ''.join([c if c.isalnum() else '_' for c in label])

def create_node(tx, label, properties):
    sanitized_label = sanitize_label(label)
    props_str = ', '.join([f'{k}: ${k}' for k in properties.keys()])
    query = f"MERGE (n:{sanitized_label} {{ {props_str} }}) RETURN n"
    tx.run(query, **properties)

def create_relationship(tx, subject, relation, object_):
    subject_label = sanitize_label(subject['label'])
    subject_props = {k: str(v) for k, v in subject.items() if k != 'label'}
    object_label = sanitize_label(object_['label'])
    object_props = {k: str(v) for k, v in object_.items() if k != 'label'}
    relation_label = sanitize_label(relation['label'])

    # Create subject node
    create_node(tx, subject_label, subject_props)
    
    # Create object node
    create_node(tx, object_label, object_props)
    
    # Create relationship
    subject_props_str = ', '.join([f'{k}: ${k}' for k in subject_props.keys()])
    object_props_str = ', '.join([f'{k}: ${k}' for k in object_props.keys()])
    query = f"""
    MATCH (a:{subject_label} {{ {subject_props_str} }})
    MATCH (b:{object_label} {{ {object_props_str} }})
    MERGE (a)-[r:{relation_label}]->(b)
    RETURN r
    """
    tx.run(query, **subject_props, **object_props)

def process_triplets(triplets):
    with driver.session() as session:
        for triplet in triplets:
            subject = triplet['subject']
            relation = triplet['relation']
            object_ = triplet['object']
            session.write_transaction(create_relationship, subject, relation, object_)

def main():
    input_file = './../../gollie_filtered_merged.jsonl'
    with jsonlines.open(input_file) as reader:
        for obj in reader:
            triplets = obj.get('triplets', [])
            for triplet_data in triplets:
                triplet_content = triplet_data.get('content', [])
                process_triplets(triplet_content)

if __name__ == "__main__":
    main()


/var/folders/04/b6fhd4q53dxfw8f2789k3sp00000gn/T/ipykernel_14083/1228926279.py:50: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship, subject, relation, object_)


TypeError: 'int' object is not iterable